- 전처리 어떻게 할까?

[ARTPRICE]
--> Title : 유지
--> Author
1. 순서대로 1,2,3,...,29,30 이렇게 붙이면?
--> year
1. 결측치 있는 것은 어떻게?
--> standard(그림크기)
1. 가로*세로 곱한 값을 가져와서 '넓이'로 만든다.
2. 결측치 있는 것은 drop?
--> genre
1. 회화 / 판화 / 서예 / 서예-현판 / 입체 / 공예 / 사진-미디어 / 기타 로 구분하여 각각에 0,1,2,3... 식의 값을 부여
--> technique
1. 요소들이 'AAAA, BBBB, CCCC' 형태로 되어 있는 것들이 많아서 이것들을 다 분리해야 함
2. 분리한 이후 각 요소들을 포함하는지의 여부를 one-hot encoding으로 정리
3. 결측치 있는 것은 어떻게?
--> source
1. **옥션, **옥션 온라인으로 구분됨

1-1. 각각의 source를 그냥 0,1,2...로 값을 준다.

1-2. 온라인 여부를 따로 column을 만들고, K옥션 온라인과 K옥션을 통합하여 전처리한다.

--> price
1. 원화 표시, 콤마 표시 제거

[Economic]
- 주가, 비트코인, 유가
: 거래 당일날의 가격에 해당하는 종가 가격을 찾아 join
- 부동산 가격
: 거래 당월에 해당하는 가격상승 값을 찾아 join

[Correlation]
- 각 요소들과 price와의 상관관계를 linechart, heatmap 등으로 표현하여 시각화


In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df = pd.read_excel('./crawling_files/artprice_total.xlsx', engine='openpyxl')
df.head()

,title,author,year,standard,genre,technique,source,date,price
0,Duet,김환기,NaN,79.5×56.7cm,판화,"오프셋 프린트, 스크린 프린트(사후판화)",아트데이옥션 온라인,2021/06/30,"￦900,000"
1,27-VIII-70 #186,김환기,2015,100×72.7cm,판화,"피그먼트 프린트, 디아섹",서울옥션 온라인,2021/06/23,"￦850,000"
2,무제,김환기,1969,120.5×85.5cm,회화,코튼에 유채,K옥션,2021/06/23,"￦680,000,000"
3,27-XI-71 #211,김환기,1971,176.3×126.3cm,회화,면에 유채,서울옥션,2021/06/22,"￦3,050,000,000"
4,25-Ⅰ-68 Ⅳ,김환기,1968,58×37.4cm,회화,신문에 유채,서울옥션,2021/06/22,"￦72,000,000"


In [3]:
df.isnull().sum()

title           0
author          0
year         1607
standard        3
genre           0
technique      89
source          0
date            0
price           0
dtype: int64

# ARTPRICE

## Author

김환기 : 0, 
이우환 : 1, 
박서보 : 2, 
정상화 : 3, 
김창열 : 4, 
박수근 : 5, 
천경자 : 6, 
이중섭 : 7, 
윤형근 : 8, 
이대원 : 9, 
유영국 : 10, 
장욱진 : 11, 
김종학 : 12, 
백남준 : 13, 
오치균 : 14, 
하종현 : 15, 
도상봉 : 16, 
이강소 : 17, 
이왈종 : 18, 
정선 : 19, 
김홍도 : 20, 
김태호 : 21, 
이상범 : 22, 
남관 : 23, 
이응노 : 24, 
전광영 : 25, 
김정희 : 26, 
서세옥 : 27, 
이성자 : 28, 
김기창 : 29

In [4]:
df['author'] = df['author'].map({'김환기' : 0, '이우환' : 1, '박서보' : 2, '정상화' : 3, '김창열' : 4, '박수근' : 5, '천경자' : 6, 
                                        '이중섭' : 7, '윤형근' : 8, '이대원' : 9, '유영국' : 10, '장욱진' : 11, '김종학' : 12, '백남준' : 13, 
                                        '오치균' : 14, '하종현' : 15, '도상봉' : 16, '이강소' : 17, '이왈종' : 18, '정선' : 19, '김홍도' : 20, 
                                        '김태호' : 21, '이상범' : 22, '남관' : 23, '이응노' : 24, '전광영' : 25, '김정희' : 26, '서세옥' : 27, 
                                        '이성자' : 28, '김기창' : 29})
df = df.astype({'author':'int'})

In [5]:
df.head()

,title,author,year,standard,genre,technique,source,date,price
0,Duet,0,NaN,79.5×56.7cm,판화,"오프셋 프린트, 스크린 프린트(사후판화)",아트데이옥션 온라인,2021/06/30,"￦900,000"
1,27-VIII-70 #186,0,2015,100×72.7cm,판화,"피그먼트 프린트, 디아섹",서울옥션 온라인,2021/06/23,"￦850,000"
2,무제,0,1969,120.5×85.5cm,회화,코튼에 유채,K옥션,2021/06/23,"￦680,000,000"
3,27-XI-71 #211,0,1971,176.3×126.3cm,회화,면에 유채,서울옥션,2021/06/22,"￦3,050,000,000"
4,25-Ⅰ-68 Ⅳ,0,1968,58×37.4cm,회화,신문에 유채,서울옥션,2021/06/22,"￦72,000,000"


## Standard

가로*세로 곱한 값을 가져와서 '넓이'로 만든다.
결측치 있는 것은 drop?

In [6]:
df[df['standard'].isnull()]

,title,author,year,standard,genre,technique,source,date,price
2980,Burunt Umber & Ultramarine Blue,8,NaN,NaN,회화,NaN,서울옥션 블루,2019/11/21,"￦90,000,000"
6168,하경산수,22,NaN,NaN,회화,종이에 수묵담채,마이아트옥션,2021/03/04,"￦16,000,000"
7013,고추,24,1979,NaN,회화,종이에 수묵담채,마이아트옥션,2017/05/22,"￦9,000,000"


결측치 데이터 3개 있어서 drop함

In [7]:
df.dropna(subset=['standard'], inplace=True)
df[df['standard'].isnull()]

,title,author,year,standard,genre,technique,source,date,price


데이터는 대개 다음과 같은 형태로 구성된다.

1. {숫자}x{숫자}cm : 대부분
2. {숫자}x{숫자}, {숫자}x{숫자}....
3. 지름 {숫자}cm
4. 지름 {숫자}x{숫자}

In [8]:
df = df[~df['standard'].str.contains('☓')]
df = df[~df['standard'].str.contains(':')]
df = df[~df['standard'].str.contains('지름')]
df = df[~df['standard'].str.contains('a.')]
df = df[~df['standard'].str.contains(',')]
df = df[~df['standard'].str.contains('외')]
df = df[~df['standard'].str.contains("\(")]
df = df[~df['standard'].str.contains("/")]
df = df[df['standard'].str.contains("×")]
df.reset_index(drop=True, inplace=True)
df

,title,author,year,standard,genre,technique,source,date,price
0,Duet,0,NaN,79.5×56.7cm,판화,"오프셋 프린트, 스크린 프린트(사후판화)",아트데이옥션 온라인,2021/06/30,"￦900,000"
1,27-VIII-70 #186,0,2015,100×72.7cm,판화,"피그먼트 프린트, 디아섹",서울옥션 온라인,2021/06/23,"￦850,000"
2,무제,0,1969,120.5×85.5cm,회화,코튼에 유채,K옥션,2021/06/23,"￦680,000,000"
3,27-XI-71 #211,0,1971,176.3×126.3cm,회화,면에 유채,서울옥션,2021/06/22,"￦3,050,000,000"
4,25-Ⅰ-68 Ⅳ,0,1968,58×37.4cm,회화,신문에 유채,서울옥션,2021/06/22,"￦72,000,000"
...,...,...,...,...,...,...,...,...,...
8177,독수리,29,1940년대,116×71cm,회화,종이에 수묵담채,아트데이옥션 온라인,2016/11/29,"￦7,000,000"
8178,군상,29,1940년대,14.8×21.8cm,회화,"종이에 먹, 연필",아트데이옥션 온라인,2016/11/29,"￦340,000"
8179,부엉이,29,1940년대,65×62.5cm,판화,석판화,K옥션 온라인,2016/11/29,"￦300,000"
8180,삽화,29,1940년대,13×17.5cm,회화,종이에 수묵,에이옥션 온라인,2016/11/23,"￦140,000"


In [9]:
df['area']=0
for j in range(len(df)):
    text = df['standard'][j]
    height = text.split('×')[0]
    width = text.split('×')[1]
    height = float(height)
    if 'cm' in width:
        width = float(width[:-2])
    else:
        width = float(width)
    df['area'][j] = height*width

<ipython-input-9-849df69f0419>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['area'][j] = height*width


## genre

In [10]:
df = df.replace(['회화', '판화', '서예', '서예-현판', '기타', '입체', '공예', '사진/미디어', '공예-도자기'],[0,1,2,3,4,5,6,7,8])
df.head()

,title,author,year,standard,genre,technique,source,date,price,area
0,Duet,0,NaN,79.5×56.7cm,1,"오프셋 프린트, 스크린 프린트(사후판화)",아트데이옥션 온라인,2021/06/30,"￦900,000",4507
1,27-VIII-70 #186,0,2015,100×72.7cm,1,"피그먼트 프린트, 디아섹",서울옥션 온라인,2021/06/23,"￦850,000",7270
2,무제,0,1969,120.5×85.5cm,0,코튼에 유채,K옥션,2021/06/23,"￦680,000,000",10302
3,27-XI-71 #211,0,1971,176.3×126.3cm,0,면에 유채,서울옥션,2021/06/22,"￦3,050,000,000",22266
4,25-Ⅰ-68 Ⅳ,0,1968,58×37.4cm,0,신문에 유채,서울옥션,2021/06/22,"￦72,000,000",2169


## technique

바탕재 : 목탄, 신문지, 종이, 직물, 캔버스, 하드보드, 한지, 도자기, 세라믹, 테라코타, 메소나이트, 나무, 낙엽, 은지

재료 : 과슈, 마카, 매직, 사인펜, 색연필, 수묵, 수묵담채, 수채, 연필, 유화, 잉크, 채색, 펜, 혼합재료, 안료, 그라파이트, 아크릴릭, 콩테, 흑연, 콜라주, 먹, 수묵채색, 새김, 크레파스, 에나멜, 오일파스텔

프린트, 스크린, 부채, 포스터, 탁본, 인쇄, 지본묵서, 판화, 접선, 장지, 신문 = 종이
카드보드 = 하드보드
드라이포인트,아쿼틴트, 에칭,믹소그라피아, 믹소그라피, 믹소그래프  = 금속
디아섹 = 아크릴
천, 비단, 세리그라프, 에코백, 면, 견, 마포, 마대, 모시, 린넨, 금견, 코튼, = 직물
도자, 세라믹 = 도자기
목판인쇄본 = 나무
켄버스 = 캔버스

유채 = 유화+채색


In [11]:
tech_dictionary = {'프린트': '종이', '스크린':'종이', '부채': '종이', '탁본' : '종이', '인쇄':'종이', '지본묵서': '종이',
             '판화': '종이', '접선':'종이', '지본수묵':'종이', '포스터':'종이','장지': '종이', '신문': '종이', '지도':'종이', 
            
            '카드보드':'하드보드', 
                   
             '드라이포인트': '금속', '아쿼틴트': '금속', '에칭': '금속','믹소그라피아': '금속','믹소그라피': '금속',
                   
            '믹소그래프': '금속','디아섹': '아크릴', 
            '천': '직물', '비단': '직물', '세리그라프': '직물','세리그래프':'직물','에코백': '직물', 
            '면': '직물', '견': '직물', '마포': '직물', '마대': '직물', '모시': '직물', '린넨': '직물', '금견': '직물', 
            '코튼': '직물', 
            '도자에': '세라믹', '도자기': '세라믹', '목판인쇄본': '나무', '켄버스':'캔버스',
            '그라파이트' : '흑연',
            '나뭇잎' : '낙엽',  
            '먹' : '수묵',
            '수묵채색' : '수묵담채',
            '유채':'유화 채색'}

tech_dictionary.items()

def tech_replace(text, dic):
    for i, j in dic.items():
        text = text.replace(i,j)
    return text

for i in range(len(df)):
    text = str(df['technique'][i])
#     print("Previous : ", text)
    new_text = tech_replace(text, tech_dictionary)
#     print(new_text)
    df['technique'][i] = new_text

<ipython-input-11-f9c270990f23>:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['technique'][i] = new_text


In [12]:
base_material = ['목탄', '종이',  '직물',  '캔버스', '하드보드', '한지', 
                 '세라믹', '테라코타', '메소나이트', '나무', '낙엽', '은지', '금속']

material = ['과슈', '마카', '매직', '사인펜', '색연필', '수묵', '수묵담채', '수채', '연필', '유화', '잉크', 
            '채색', '펜', '혼합재료', '안료', '그라파이트', '아크릴', '콩테', '콜라주', '새김', 
            '크레파스', '에나멜', '오일파스텔']
arr = ['목탄', '종이', '직물', '캔버스', '하드보드', '한지','세라믹', '테라코타', '메소나이트', '나무', '낙엽', '은지', '금속',
       '과슈', '마카', '매직', '사인펜', '색연필', '수묵', '수묵담채', '수채', '연필', '유화', '유채', '잉크', '채색', '펜', 
       '혼합재료', '안료', '흑연', '아크릴', '콩테', '콜라주', '새김', '크레파스', '에나멜', '오일파스텔']
tech_iter = ['base_charcoal', 'base_paper', 'base_fiber', 'base_canvas', 'base_hardboard', 'base_hanji', 'base_ceramic',
            'base_terracotta', 'base_masonite', 'base_wood', 'base_leaf', 'base_silverpaper', 'base_metal',
            'mate_gouache', 'mate_maca', 'mate_magic', 'mate_signpen', 'mate_colorpencil', 'mate_korean_ink', 'mate_korean_ink_and_color',
            'mate_watercolor', 'mate_pencil', 'mate_oil', 'mate_oil_and_color', 'mate_ink', 'mate_color', 'mate_pen', 'mate_mixture',
            'mate_pigment', 'mate_graphite', 'mate_acrylic', 'mate_conte', 'mate_collage', 'mate_engrave', 'mate_crayon', 'mate_enamel',
            'mate_oilpastel']
print(len(arr))
print(len(tech_iter))

37
37


In [13]:
df['base_charcoal'] = 0
df['base_paper'] = 0
df['base_fiber'] = 0
df['base_canvas'] = 0
df['base_hardboard'] = 0
df['base_hanji'] = 0
df['base_ceramic'] = 0
df['base_terracotta'] = 0
df['base_masonite'] = 0
df['base_wood'] = 0
df['base_leaf'] = 0
df['base_silverpaper'] = 0
df['base_metal'] = 0

df['mate_gouache'] = 0
df['mate_maca'] = 0
df['mate_magic'] = 0
df['mate_signpen'] = 0
df['mate_colorpencil'] = 0
df['mate_korean_ink'] = 0
df['mate_korean_ink_and_color'] = 0
df['mate_watercolor'] = 0
df['mate_pencil'] = 0
df['mate_oil'] = 0
df['mate_oil_and_color'] = 0
df['mate_ink'] = 0
df['mate_color'] = 0
df['mate_pen'] = 0
df['mate_mixture'] = 0
df['mate_pigment'] = 0 #안료
df['mate_graphite'] = 0
df['mate_acrylic'] = 0
df['mate_conte'] = 0
df['mate_collage'] = 0
df['mate_engrave'] = 0 #새김
df['mate_crayon'] = 0 #크레파스
df['mate_enamel'] = 0
df['mate_oilpastel'] = 0

In [14]:
df_copy = df.copy()

In [15]:
for i in range(0, len(df_copy)):
    text = str(df_copy['technique'][i])
    for j in range(0, len(arr)):
        if arr[j] in text:
            df_copy[tech_iter[j]][i]=1
            continue

<ipython-input-15-f087a86ac1fc>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_copy[tech_iter[j]][i]=1


In [16]:
df_copy['base_paper'].value_counts()

1    5342
0    2840
Name: base_paper, dtype: int64

In [17]:
for word in tech_iter:
    print(word)
    print(df_copy[word].value_counts())

base_charcoal
0    8160
1      22
Name: base_charcoal, dtype: int64
base_paper
1    5342
0    2840
Name: base_paper, dtype: int64
base_fiber
0    7428
1     754
Name: base_fiber, dtype: int64
base_canvas
0    6635
1    1547
Name: base_canvas, dtype: int64
base_hardboard
0    8155
1      27
Name: base_hardboard, dtype: int64
base_hanji
0    7863
1     319
Name: base_hanji, dtype: int64
base_ceramic
0    8168
1      14
Name: base_ceramic, dtype: int64
base_terracotta
0    8170
1      12
Name: base_terracotta, dtype: int64
base_masonite
0    8180
1       2
Name: base_masonite, dtype: int64
base_wood
0    8118
1      64
Name: base_wood, dtype: int64
base_leaf
0    8168
1      14
Name: base_leaf, dtype: int64
base_silverpaper
0    8176
1       6
Name: base_silverpaper, dtype: int64
base_metal
0    8089
1      93
Name: base_metal, dtype: int64
mate_gouache
0    8051
1     131
Name: mate_gouache, dtype: int64
mate_maca
0    8182
Name: mate_maca, dtype: int64
mate_magic
0    8171
1      11
Nam

In [18]:
df = df_copy

## source

K옥션 = 0 0
K옥션 온라인 = 0 1
서울옥션(홍콩, 블루) = 1 0
서울옥션 온라인 = 1 1
아트데이옥션 = 2 0
아트데이옥션 온라인 = 2 1
아이옥션 = 3 0
아이옥션 온라인 = 3 1
마이아트옥션 = 4 0
마이아트옥션 온라인 = 4 1
에이옥션 온라인 = 5 1
칸옥션 = 6 0
꼬모옥션 온라인 = 7 1

In [19]:
df['source'].value_counts()

K옥션 온라인       3101
K옥션           1378
서울옥션           870
아트데이옥션 온라인     701
에이옥션 온라인       509
서울옥션 온라인       449
아이옥션           253
서울옥션 홍콩        236
서울옥션 블루        145
아이옥션 온라인       132
마이아트옥션         126
아트데이옥션         102
칸옥션            101
꼬모옥션 온라인        68
마이아트옥션 온라인      11
Name: source, dtype: int64

In [20]:
df['online'] = 0
for i in range(0, len(df)):
    if '온라인' in df['source'][i]:
        df['online'][i] = 1

<ipython-input-20-daf944d43a37>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['online'][i] = 1


In [21]:
df['online'].value_counts()

1    4971
0    3211
Name: online, dtype: int64

In [22]:
df = df.replace(['K옥션 온라인', 'K옥션'],[0,0])
df = df.replace(['서울옥션', '서울옥션 온라인', '서울옥션 홍콩', '서울옥션 블루'],[1, 1, 1, 1])
df = df.replace(['아트데이옥션 온라인', '아트데이옥션'],[2, 2])
df = df.replace(['아이옥션 온라인', '아이옥션'],[3, 3])
df = df.replace(['마이아트옥션 온라인', '마이아트옥션'],[4, 4])
df = df.replace(['에이옥션 온라인', '칸옥션', '꼬모옥션 온라인'],[5,6,7])

df['source'].value_counts()

0    4479
1    1700
2     803
5     509
3     385
4     137
6     101
7      68
Name: source, dtype: int64

## price

In [23]:
df['price']

0             ￦900,000
1             ￦850,000
2         ￦680,000,000
3       ￦3,050,000,000
4          ￦72,000,000
             ...      
8177        ￦7,000,000
8178          ￦340,000
8179          ￦300,000
8180          ￦140,000
8181        ￦1,800,000
Name: price, Length: 8182, dtype: object

In [24]:
for i in range(0, len(df)):
    text = df['price'][i]
    text = text.replace('￦','')
    text=  text.replace(',', '')
    df['price'][i] = text
df['price']

<ipython-input-24-01d8861cef3a>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['price'][i] = text


0           900000
1           850000
2        680000000
3       3050000000
4         72000000
           ...    
8177       7000000
8178        340000
8179        300000
8180        140000
8181       1800000
Name: price, Length: 8182, dtype: object

## Year

결측치는 해당 작가가 출품한 작품이 제일 많은 해의 연도를 삽입할 예정

~ 처리된 것은 맨 마지막 값을 제작연도로 계산함. Ex.1993~95 면 1995가 됨

월/일이 표시된 것은 해당 연도를 제작연도로 계산함. Ex.1997.10면 1997이 됨

, 처리된 것은 큰 값을 제작연도로 계산함 Ex. 1995, 1996이면 1996이 됨

~년대 인 경우 해당 시대의 맨 마지막 연도로 계산함. Ex. 1980년대 면 1989년이 됨

~년경 인 경우 해당 연도를 제작연도로 계산함 Ex. 1991년경 이면 1991년이 됨

연도에 문자가 섞인 경우 문자를 제거 Ex. b.1998면 1998년이 됨

연도 미상인 것들이 많고 작가의 생몰년도가 정확한 경우 작가의 사망연도를 제작연도로 함. Ex. 정선의 사망연도가 1759년이므로 정선 작품 중 제작연도를 알 수 없는 것들은 모두 1759년으로 함.


In [68]:
row_count = 0
for author in range(30):
    di = dict(df[df['author']==author]['year'].value_counts())
    max_year = str(max(di, key=di.get))
    if author==19:
        max_year = 1759 # 정선 사망 연도가 1759
    elif author==26:
        max_year = 1856 # 김정희 사망 연도가 1856
    elif '년대' in max_year:
        max_year = int(max_year[:3]+'9')
    else:
        max_year = int(max_year)
    print(author, max_year)
    for _ in range(0, len(df[df['author']==author]['year'])):
        year_text = str(df[df['author']==author]['year'][row_count])
        if len(year_text)==3:
            year = max_year
        elif '~' in year_text:
            year = year_text.split('~')[0][:2]+year_text.split('~')[1][:2]
        elif '/' in year_text:
            year = year_text[:4]
        elif '.' in year_text:
            year = year_text[:4]
        elif ',' in year_text:
            year = max(list(year_text.split(',')))
        elif '년대 초반' in year_text:
            year = year_text[:3]+'2'    
        elif '년대 중반' in year_text:
            year = year_text[:3]+'6'
        elif '년대 후반' in year_text:
            year = year_text[:3]+'9'
        elif '년대' in year_text:
            year = year_text[:3]+'9'
        elif '년경' in year_text:
            year = year_text[:4]
        elif 'a' in year_text or 'b' in year_text or 'c' in year_text:
            year = year_text[2:]
        elif '세기' in year_text:
            year = int(year_text.split('세기')[0])*100+150
        elif '(' in year_text:
            year = year_text[:4]
        else:
            year = int(year_text)
        df['year'][row_count] = year
        row_count+=1

0 1968


<ipython-input-68-ca4fa6ed863b>:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['year'][row_count] = year


1 2012
2 2006
3 1990
4 2016
5 2010
6 2006
7 1959
8 1995
9 1996
10 1989
11 1979
12 2003
13 1988
14 2012
15 1995
16 1976
17 2007
18 2016
19 1759
20 1786
21 2016
22 1963
23 1975
24 1978
25 2007
26 1856
27 1989
28 1960
29 1981


In [72]:
df.drop(['title', 'standard', 'technique'], axis='columns', inplace=True)

In [74]:
df.to_excel('preprocessing_art_feature.xlsx')

In [77]:
df.head()

,author,year,genre,source,date,price,area,base_charcoal,base_paper,base_fiber,...,mate_pigment,mate_graphite,mate_acrylic,mate_conte,mate_collage,mate_engrave,mate_crayon,mate_enamel,mate_oilpastel,online
0,0,1968,1,2,2021/06/30,900000,4507,0,1,0,...,0,0,0,0,0,0,0,0,0,1
1,0,2015,1,1,2021/06/23,850000,7270,0,1,0,...,0,0,1,0,0,0,0,0,0,1
2,0,1969,0,0,2021/06/23,680000000,10302,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3,0,1971,0,1,2021/06/22,3050000000,22266,0,0,1,...,0,0,0,0,0,0,0,0,0,0
4,0,1968,0,1,2021/06/22,72000000,2169,0,1,0,...,0,0,0,0,0,0,0,0,0,0


# Merging Economic Feature

주식 : KOSPI(Clear)

BTC : Upbit(데이터 기간이 일치하지 않음) -> Coinbase?

부동산 : KB월간주택가격동향

환율 : USD_KRW(Clear)

원유 : WTI유 선물내역(Clear)

기존 df에 merge할 예정

In [141]:
df = pd.read_excel('preprocessing_art_feature.xlsx', engine='openpyxl')

In [142]:
df['date'] = pd.to_datetime(df['date'])

## Merge KOSPI

In [150]:
kospi = pd.read_csv('economic_historical_data/KOSPI_20160101~20211112.csv', encoding='cp949')
kospi.head()

,일자,종가,대비,등락률,시가,고가,저가,거래량,거래대금,상장시가총액
0,2021/11/12,2968.80,43.88,1.50,2945.95,2973.84,2938.27,665733196,12472956374638,2185439467790075
1,2021/11/11,2924.92,-5.25,-0.18,2911.41,2925.19,2903.72,572859058,13468395643103,2153082343088190
2,2021/11/10,2930.17,-32.29,-1.09,2947.46,2958.61,2924.36,570931752,9663221463613,2157022729552783
3,2021/11/09,2962.46,2.26,0.08,2958.15,2987.28,2949.17,708773471,9684977252764,2180787959345307
4,2021/11/08,2960.20,-9.07,-0.31,2965.36,2968.18,2930.90,560205356,10388089493103,2178020380431059


In [151]:
kospi.drop(['대비','등락률','시가','고가','저가','거래량','거래대금','상장시가총액'], axis='columns', inplace=True)

In [152]:
kospi = kospi.rename(columns={'일자': 'date', '종가':'KOSPI_Close'})
kospi['date'] = pd.to_datetime(kospi['date'])

In [153]:
kospi

,date,KOSPI_Close
0,2021-11-12,2968.80
1,2021-11-11,2924.92
2,2021-11-10,2930.17
3,2021-11-09,2962.46
4,2021-11-08,2960.20
...,...,...
1233,2016-11-07,1997.58
1234,2016-11-04,1982.02
1235,2016-11-03,1983.80
1236,2016-11-02,1978.94


In [154]:
df = pd.merge(df, kospi, left_on='date', right_on='date', how='left')
df.drop(['Unnamed: 0'],axis='columns',inplace=True)

In [155]:
df

,author,year,genre,source,date,price,area,base_charcoal,base_paper,base_fiber,...,mate_graphite,mate_acrylic,mate_conte,mate_collage,mate_engrave,mate_crayon,mate_enamel,mate_oilpastel,online,KOSPI_Close
0,0,1968,1,2,2021-06-30,900000,4507,0,1,0,...,0,0,0,0,0,0,0,0,1,3296.68
1,0,2015,1,1,2021-06-23,850000,7270,0,1,0,...,0,1,0,0,0,0,0,0,1,3276.19
2,0,1969,0,0,2021-06-23,680000000,10302,0,0,1,...,0,0,0,0,0,0,0,0,0,3276.19
3,0,1971,0,1,2021-06-22,3050000000,22266,0,0,1,...,0,0,0,0,0,0,0,0,0,3263.88
4,0,1968,0,1,2021-06-22,72000000,2169,0,1,0,...,0,0,0,0,0,0,0,0,0,3263.88
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8177,29,1949,0,2,2016-11-29,7000000,8236,0,1,0,...,0,0,0,0,0,0,0,0,1,1978.39
8178,29,1949,0,2,2016-11-29,340000,322,0,1,0,...,0,0,0,0,0,0,0,0,1,1978.39
8179,29,1949,1,0,2016-11-29,300000,4062,0,1,0,...,0,0,0,0,0,0,0,0,1,1978.39
8180,29,1949,0,5,2016-11-23,140000,227,0,1,0,...,0,0,0,0,0,0,0,0,1,1987.95


## Merge Oil

In [156]:
oil = pd.read_csv('economic_historical_data/WTI유 선물 내역.csv')
oil.head()

,날짜,종가,오픈,고가,저가,거래량,변동 %
0,2021/11/12,80.72,81.36,81.59,79.80,-,-1.07%
1,2021/11/11,81.59,81.41,82.33,80.20,-,0.31%
2,2021/11/10,81.34,84.53,84.97,80.81,555.22K,-3.34%
3,2021/11/9,84.15,82.16,84.63,81.78,504.83K,2.71%
4,2021/11/8,81.93,81.13,82.67,81.05,393.95K,0.81%


In [157]:
oil.drop(['오픈','고가','저가','거래량','변동 %'], axis='columns', inplace=True)

In [158]:
oil = oil.rename(columns={'날짜': 'date', '종가':'WTI_Close'})
oil['date'] = pd.to_datetime(oil['date'])

In [159]:
oil

,date,WTI_Close
0,2021-11-12,80.72
1,2021-11-11,81.59
2,2021-11-10,81.34
3,2021-11-09,84.15
4,2021-11-08,81.93
...,...,...
1543,2016-01-08,33.16
1544,2016-01-07,33.27
1545,2016-01-06,33.97
1546,2016-01-05,35.97


In [160]:
df = pd.merge(df, oil, left_on='date', right_on='date', how='left')
df

,author,year,genre,source,date,price,area,base_charcoal,base_paper,base_fiber,...,mate_acrylic,mate_conte,mate_collage,mate_engrave,mate_crayon,mate_enamel,mate_oilpastel,online,KOSPI_Close,WTI_Close
0,0,1968,1,2,2021-06-30,900000,4507,0,1,0,...,0,0,0,0,0,0,0,1,3296.68,73.47
1,0,2015,1,1,2021-06-23,850000,7270,0,1,0,...,1,0,0,0,0,0,0,1,3276.19,73.08
2,0,1969,0,0,2021-06-23,680000000,10302,0,0,1,...,0,0,0,0,0,0,0,0,3276.19,73.08
3,0,1971,0,1,2021-06-22,3050000000,22266,0,0,1,...,0,0,0,0,0,0,0,0,3263.88,73.06
4,0,1968,0,1,2021-06-22,72000000,2169,0,1,0,...,0,0,0,0,0,0,0,0,3263.88,73.06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8177,29,1949,0,2,2016-11-29,7000000,8236,0,1,0,...,0,0,0,0,0,0,0,1,1978.39,45.23
8178,29,1949,0,2,2016-11-29,340000,322,0,1,0,...,0,0,0,0,0,0,0,1,1978.39,45.23
8179,29,1949,1,0,2016-11-29,300000,4062,0,1,0,...,0,0,0,0,0,0,0,1,1978.39,45.23
8180,29,1949,0,5,2016-11-23,140000,227,0,1,0,...,0,0,0,0,0,0,0,1,1987.95,47.96


## Merging Exchange Rate

In [164]:
rate = pd.read_csv('economic_historical_data/USD_KRW 내역.csv')
rate

,날짜,종가,오픈,고가,저가,변동 %
0,2021/11/12,"1,179.94","1,181.03","1,182.35","1,176.45",-0.21%
1,2021/11/11,"1,182.39","1,185.15","1,187.30","1,178.17",-0.22%
2,2021/11/10,"1,185.04","1,178.51","1,186.15","1,176.82",0.56%
3,2021/11/9,"1,178.40","1,180.95","1,182.10","1,173.36",-0.21%
4,2021/11/8,"1,180.84","1,181.51","1,186.74","1,178.93",-0.02%
...,...,...,...,...,...,...
1526,2016/1/7,"1,197.29","1,200.52","1,204.70","1,193.16",-0.27%
1527,2016/1/6,"1,200.52","1,190.81","1,203.80","1,187.59",0.82%
1528,2016/1/5,"1,190.81","1,190.35","1,192.80","1,184.40",0.04%
1529,2016/1/4,"1,190.35","1,175.66","1,193.23","1,175.65",1.27%


In [165]:
rate.drop(['오픈','고가','저가', '변동 %'], axis='columns', inplace=True)
rate = rate.rename(columns={'날짜': 'date', '종가':'rate_Close'})
rate['date'] = pd.to_datetime(rate['date'])

In [166]:
df = pd.merge(df, rate, left_on='date', right_on='date', how='left')
df

,author,year,genre,source,date,price,area,base_charcoal,base_paper,base_fiber,...,mate_conte,mate_collage,mate_engrave,mate_crayon,mate_enamel,mate_oilpastel,online,KOSPI_Close,WTI_Close,rate_Close
0,0,1968,1,2,2021-06-30,900000,4507,0,1,0,...,0,0,0,0,0,0,1,3296.68,73.47,"1,130.48"
1,0,2015,1,1,2021-06-23,850000,7270,0,1,0,...,0,0,0,0,0,0,1,3276.19,73.08,"1,135.85"
2,0,1969,0,0,2021-06-23,680000000,10302,0,0,1,...,0,0,0,0,0,0,0,3276.19,73.08,"1,135.85"
3,0,1971,0,1,2021-06-22,3050000000,22266,0,0,1,...,0,0,0,0,0,0,0,3263.88,73.06,"1,133.57"
4,0,1968,0,1,2021-06-22,72000000,2169,0,1,0,...,0,0,0,0,0,0,0,3263.88,73.06,"1,133.57"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8177,29,1949,0,2,2016-11-29,7000000,8236,0,1,0,...,0,0,0,0,0,0,1,1978.39,45.23,"1,167.76"
8178,29,1949,0,2,2016-11-29,340000,322,0,1,0,...,0,0,0,0,0,0,1,1978.39,45.23,"1,167.76"
8179,29,1949,1,0,2016-11-29,300000,4062,0,1,0,...,0,0,0,0,0,0,1,1978.39,45.23,"1,167.76"
8180,29,1949,0,5,2016-11-23,140000,227,0,1,0,...,0,0,0,0,0,0,1,1987.95,47.96,"1,181.89"


## Merging estate

In [169]:
estate_rate = {
    2016:{
        1:94.5, 2:94.6, 3:94.6, 4:94.7,
        5:94.7, 6:94.8, 7:94.9, 8:95.1,
        9:95.2, 10:95.4, 11:95.7, 12:95.7
    },
    2017:{
        1:95.7, 2:95.8, 3:95.8, 4:95.8,
        5:95.9, 6:96.0, 7:96.3, 8:96.5,
        9:96.6, 10:96.7, 11:96.8, 12:96.9,
    },
    2018:{
        1:97.1, 2:97.3, 3:97.5, 4:97.7,
        5:97.8, 6:97.9, 7:98.0, 8:98.2,
        9:99.2, 10:99.8, 11:99.9, 12:100.0,
    },
    2019:{
        1:100.0, 2:99.9, 3:99.83, 4:99.7,
        5:99.6, 6:99.5, 7:99.5, 8:99.6,
        9:99.6, 10:99.7, 11:99.9, 12:100.2,
    },
    2020:{
        1:100.6, 2:100.9, 3:101.5, 4:101.7,
        5:101.9, 6:102.4, 7:103.3, 8:104.1,
        9:104.9, 10:105.6, 11:107.1, 12:108.6,
    },
    2021:{
        1:109.9, 2:111.4, 3:112.8, 4:114.0,
        5:115.1, 6:116.6, 7:118.0, 8:119.8,
        9:121.6, 10:122.9
    }
}

In [191]:
def find_month(i, year):
    if df['date'][i].month == 1:
        return estate_rate[year][1]
    elif df['date'][i].month == 2:
        return estate_rate[year][2]
    elif df['date'][i].month == 3:
        return estate_rate[year][3]
    elif df['date'][i].month == 4:
        return estate_rate[year][4]
    elif df['date'][i].month == 5:
        return estate_rate[year][5]
    elif df['date'][i].month == 6:
        return estate_rate[year][6]
    elif df['date'][i].month == 7:
        return estate_rate[year][7]
    elif df['date'][i].month == 8:
        return estate_rate[year][8]
    elif df['date'][i].month == 9:
        return estate_rate[year][9]
    elif df['date'][i].month == 10:
        return estate_rate[year][10]
    elif df['date'][i].month == 11:
        return estate_rate[year][11]
    elif df['date'][i].month == 12:
        return estate_rate[year][12]

In [192]:
df['estate_rate'] = 0.0
for i in range(len(df)):
    if df['date'][i].year == 2016:
        year = 2016
        df['estate_rate'][i] = find_month(i, year)
    elif df['date'][i].year == 2017:
        year = 2017
        df['estate_rate'][i] = find_month(i, year)
    elif df['date'][i].year == 2018:
        year = 2018
        df['estate_rate'][i] = find_month(i, year)
    elif df['date'][i].year == 2019:
        year = 2019
        df['estate_rate'][i] = find_month(i, year)
    elif df['date'][i].year == 2020:
        year = 2020
        df['estate_rate'][i] = find_month(i, year)
    elif df['date'][i].year == 2021:
        year = 2021
        df['estate_rate'][i] = find_month(i, year)

<ipython-input-192-549c4e2cd75c>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['estate_rate'][i] = find_month(i, year)
<ipython-input-192-549c4e2cd75c>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['estate_rate'][i] = find_month(i, year)
<ipython-input-192-549c4e2cd75c>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['estate_rate'][i] = find_month(i, year)
<ipython-input-192-549c4e2cd75c>:11: SettingWithCopyWarning: 

## Merging BTC

BTC/KRW 시장 데이터의 기간이 artprice의 데이터의 기간보다 짧아서 일단 merge 보류

In [196]:
btc_kor = pd.read_csv('./economic_historical_data/BTC_upbit_daily_2016-01-01-2021-11-12.csv', encoding='cp949')
btc_kor.head(-10)

,date,시가,고가,저가,종가,누적 거래량,변화금액
0,2021-11-12,78938000.0,79100000.0,76712000.0,78165000.0,7117.165348,-762000.0
1,2021-11-11,79663000.0,79732000.0,78543000.0,78917000.0,7167.446830,-743000.0
2,2021-11-10,80813000.0,82520000.0,78510000.0,79660000.0,13173.316256,-1197000.0
3,2021-11-09,81402000.0,82700000.0,79987000.0,80857000.0,9216.260721,-546000.0
4,2021-11-08,76649000.0,81586000.0,76534000.0,81403000.0,9132.136668,4867000.0
...,...,...,...,...,...,...,...
1780,2017-10-09,5222000.0,5761000.0,5188000.0,5477000.0,8.070434,260000.0
1781,2017-10-08,5044000.0,5263000.0,4993000.0,5217000.0,145.917860,170000.0
1782,2017-10-07,4983000.0,5070000.0,4935000.0,5047000.0,4.343932,20000.0
1783,2017-10-06,4927000.0,5034000.0,4885000.0,5027000.0,25.163820,101000.0


In [199]:
df.to_excel('feature_preprocessed.xlsx')